In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import BorderlineSMOTE
import xgboost as xgb

df = pd.read_csv("Base.csv")
df.head()

,fraud_bool,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,0,0.3,0.986506,-1,25,40,0.006735,102.453711,AA,1059,...,0,1500.0,0,INTERNET,16.224843,linux,1,1,0,0
1,0,0.8,0.617426,-1,89,20,0.010095,-0.849551,AD,1658,...,0,1500.0,0,INTERNET,3.363854,other,1,1,0,0
2,0,0.8,0.996707,9,14,40,0.012316,-1.490386,AB,1095,...,0,200.0,0,INTERNET,22.730559,windows,0,1,0,0
3,0,0.6,0.475100,11,14,30,0.006991,-1.863101,AB,3483,...,0,200.0,0,INTERNET,15.215816,linux,1,1,0,0
4,0,0.9,0.842307,-1,29,40,5.742626,47.152498,AA,2339,...,0,200.0,0,INTERNET,3.743048,other,0,1,0,0


In [4]:
# Convertir variables categóricas a tipo 'category'
df['payment_type'] = df.payment_type.astype('category')
df['employment_status'] = df.employment_status.astype('category')
df['housing_status'] = df.housing_status.astype('category')
df['source'] = df.source.astype('category')
df['device_os'] = df.device_os.astype('category')

In [6]:
# Aplicar One-Hot Encoding
df = pd.get_dummies(df, drop_first=True)

In [8]:
# Separar variables predictoras (X) y variable objetivo (y)
X = df.drop(['fraud_bool'], axis=1)
y = df['fraud_bool']

In [10]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [12]:
# Verificar las formas de los conjuntos
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(700000, 47) (300000, 47) (700000,) (300000,)


In [14]:
# Aplicar Borderline-SMOTE
borderline_smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
X_train_borderline_smote, y_train_borderline_smote = borderline_smote.fit_resample(X_train, y_train)

In [16]:
# XGBoost con Borderline-SMOTE
xgb_borderline_smote = xgb.XGBClassifier(random_state=42)
xgb_borderline_smote.fit(X_train_borderline_smote, y_train_borderline_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [22]:
# Predicciones XGBoost con Borderline-SMOTE
y_pred_xgb_borderline_smote = xgb_borderline_smote.predict(X_test)
print("Matriz de confusión")
print(confusion_matrix(y_test, y_pred_xgb_borderline_smote))

Matriz de confusión
[[295705    986]
 [  2977    332]]


In [20]:
print(classification_report(y_test, y_pred_xgb_borderline_smote))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    296691
           1       0.25      0.10      0.14      3309

    accuracy                           0.99    300000
   macro avg       0.62      0.55      0.57    300000
weighted avg       0.98      0.99      0.98    300000



In [ ]:
Análisis de los Resultados:
Precisión (0): 0.99 para la clase mayoritaria, lo que indica que el modelo está muy acertado al predecir la clase no fraude.

Recall (0): 1.00, lo que significa que casi todas las instancias de la clase mayoritaria están siendo correctamente clasificadas.

Precisión (1): 0.25, lo que es bastante bajo. Esto indica que de todas las predicciones que el modelo hizo para la clase de fraude, solo el 25% fueron correctas.

Recall (1): 0.10, lo que significa que el modelo está identificando solo el 10% de los casos de fraude. Este es un recall bajo y un f1-score bajo, lo cual es preocupante.

Aunque el modelo muestra una alta precisión general y un buen rendimiento para la clase mayoritaria, el modelo no está detectando adecuadamente los fraudes. Esto es crítico en escenarios de fraude, donde el objetivo principal es identificar correctamente la clase minoritaria